## 本チュートリアルの概要
　本チュートリアルでは、「文章」を入力として与えた際に、文章が属する「カテゴリ」を出力する人工知能(AI)を作る方法について解説します。例えば「今日の天気は晴れです」という文章を与えた際に「天気」というカテゴリを、「今日の日経平均株価は？」という文章を与えた際には「経済」のカテゴリの文章だというように、文章のカテゴリを分類して出力するような人工知能です。

### 分類するカテゴリ
　本チュートリアルでは、任意の文章を与えた際に、文章を次のようなカテゴリに分類してくれる人工知能を作ります。
 
 1. 天気
 2. 交通
 3. テクノロジ
 4. 経済

### 人工知能が出力する値
　本チュートリアルで完成させる人工知能は、上記のように文章を4つのカテゴリに分類してくれるわけですが、具体的にどのように文章を分類した結果を教えてくれるのかを先に説明します。

　本チュートリアルで完成させる人工知能は、任意の文章を与えた際に、上記4つのどのカテゴリに当たるかを**確率**で返却します。 次の表の例のように、それぞれのカテゴリごとに自信度のような数値を返却してくれます。

| カテゴリ   | 確率 (自信度) |
|:---------|:------------|
| 天気      | 0.145 |
| 交通      | 0.762 |
| テクノロジ | 0.013 |
| 経済      | 0.080 |

　上の表の例の場合、カテゴリ「交通」が一番確率が高いので、人工知能が与えられた文章から「交通」のカテゴリの文章だと判断したということになります。

## 「モデル」「学習」「教師データ」
　人工知能が物事を判断するための「脳みそ」に当たる部分を**モデル**と言います。 モデルを作るためには、モデルを賢くするための教材となる**教師データ**が必要で、教師データを利用して空っぽのモデル(脳みそ)を賢くすることを**学習**と言います。

### 教師データ
　教師データは、「**データ**」と「**ラベル**(正解データとも言う)」の2つがセットで構成されます。 データは人工知能への入力となるもので、ラベルは人工知能からの出力となるものです。 今回の場合は「文章」が人工知能の入力になる「**データ**」で、文章を表す「カテゴリ」が「**ラベル**」です。

### 本チュートリアルにおける教師データ
　本チュートリアルでは、筆者が独自に作った次のようなデータを教師データとして利用し、「こういう文章の場合は、このカテゴリだよ」ということを学習させます。

#### データフォーマット
以下のような構造のテキストファイルを用意します。 各項目はカンマで区切ることとします。

| 項目番号 | 項目 |
|:--------|:----|
| 1       | ラベル : カテゴリ名 |
| 2       | データ : 文章に登場する単語 (複数をスペース区切り) |

#### 実際のデータ例
例えば「本日の天気は晴れ、最高気温は26度です」という文章が「天気」というカテゴリだということを教えたい場合は、次のような教師データを用意しました。

    天気,本日 天気 晴れ 最高気温 26度

このような行をたくさん束ねた、以下のようなテキストファイルを学習用データとして利用します。

    天気,本日 天気 晴れ 最高気温 26度
    経済,午前 日経平均株価 終値 20000円
    交通,現在 山手線 外回り 遅延 5分
    天気,週末 都心 初雪 可能性
    テクノロジ,Apple 健康分野 新サービス
    (以下略)

データ行の一番左には「カテゴリ」があり、あとにはカテゴリに関連する名詞が列挙されているように見えます。

## 教師データのベクトル化
　上で述べたように、モデルを学習させるための教師データとして、(1)文章がカテゴリ「天気」だということを表す「ラベル」 と (2)「今日の天気は晴れ、最高気温は26度です」という文章から名詞を抜き出した「データ」のセットをモデルに学習させるわけですが、文字列のままコンピュータに処理させるのは適さないので、データを別の表現に変換します。 「コンピュータに処理させるのに適した表現」とは、**数値(小数値)ベクトル**です。

### ラベル
　まずは「カテゴリ」を示す「ラベル」について説明します。 4つのカテゴリを「1」「2」「3」「4」のような数値で表したいところですが、そのようにはしません。 上で説明した「人工知能が出力する値」を思い出してください。 人工知能が最終的に出力するのは各カテゴリごとの確率です。 教師データのラベルも、この形に合わせます。

　モデルを学習させる時点において、例えば「今日の天気は晴れです、最高気温は26度です」をカテゴリ「天気」だと教育したい場合、「今日の天気は〜」の文章がカテゴリ「天気」である確率は100%です。 よって、上記4つのカテゴリの順序を「天気」「交通」「テクノロジ」「経済」とした場合、それぞれのカテゴリの確率を、次のような数列で表すことができます。

    [1, 0, 0, 0]

　このような数列の値を教師データのラベルの値とします。

### データ
　次にデータ部についても数値ベクトル化する方法について解説します。 上で示したデータ例を思い出してみましょう。 次のように「天気」というカテゴリに紐づく単語が列挙されていました。

    天気,本日 天気 晴れ 最高気温 26度

　逆に言えば「本日」「天気」「晴れ」「最高気温」「26度」という単語が文章中に出現している場合、カテゴリは「天気」である確率が高そうです。 つまり、特定の単語が含まれているか否かを「コンピュータで計算しやすい形式」つまり「数値ベクトル」で表せれば良さそうです。

　そういうことであれば、教師データに登場する全単語の数の次元の数列を考え、単語が登場する場合は「1」を、単語が登場しない場合は「0」で表現するのはどうでしょうか？ 具体的には次のようなイメージです。

　教師データに登場する全単語の次元の数列を考えます。

    [本日, 天気, 晴れ, 最高気温, 26度, 午前, 日経平均株価, 終値, 20000円, 現在, 山手線, 外回り, 遅延, 5分, (以降略)]

　そして、それぞれの単語について文章中で登場すれば「1」とし、登場しなければ「0」とします。 「本日の天気は晴れ、最高気温は26度です」の文章の場合は、次のように表すことができます。

    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, (以降略)]

「ラベル」と「データ」を数値ベクトルとして表せましたので、この2つのベクトルの関係を人工知能のモデルに学習させるのです。 つまり \[1, 1, 1, 1, 1, 0, 0, 0, (以降略)] の配列が入力された場合は \[1, 0, 0, 0] が正解だと言うことを覚えさせるのです。

## 本チュートリアル 全コード
　まずは、本チュートリアルで利用するソースコード全行を示します。 この後にソースコードの部分部分について詳しく解説しますので、今ここで内容を理解する必要はありません。 まずは「これくらいのボリュームのソースコードを作るのか」という程度に眺めてください。

<script src="https://gist.github.com/oscasierra/00d57d53ddaade973bf46a1450852acb.js"></script>

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import sys
import os
import heapq
import glob
import collections
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer

###
# 用意したデータを読み込むフェーズです。
###
# 読み込むファイルの名前
file = "data.txt"

# 各行のラベルを保持するリスト
labels = []

# 各行のデータ部を保持するリスト
words_arr  = []

with open(file) as f:
  for line in f:

    # 行をカンマで分割し、ラベル部とデータ部を分割します
    blocks = line.rstrip().split(',',1)

    # ラベル部, データ部 それぞれをリストに追加します
    labels.append(blocks[0])
    words_arr.append(blocks[1])

###
# データ部・ラベル部をそれぞれベクトル化します。
###

# データ部をベクトル化します
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words_arr)
x_data = tokenizer.texts_to_matrix(words_arr, "binary")

# ラベル部をベクトル化します
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
y_data = label_tokenizer.texts_to_matrix(labels, "binary")

###
# 学習用データと検証用データにデータを分割します。
# ここでは 9:1 の割合に分割します。
###
train_size = int(len(x_data) * 0.9)
x_train,x_test = x_data[:train_size],x_data[train_size:]
y_train,y_test = y_data[:train_size],y_data[train_size:]

##
# これから作り出す脳みその形を宣言します。
# InputLayer : モデルへの入力レイヤー (ここでは、データ部の総単語数の入力を可能にします。)
# Dense      : モデルの出力。 第1引数で出力の次元数(＝ラベルの総数)を指定します。
##
model = Sequential()
model.add(InputLayer(input_shape=(x_train.shape[1],)))
model.add(Dense(y_train.shape[1], activation='softmax'))

##
# モデルをコンパイルします。
# まだ何も学習していない空っぽの脳みそを生成するイメージです。
##
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

###
# model.fit() を実行することにより、モデルの学習を行います。
# batch_size : 1回の学習での入力サイズ
# epochs : 学習する回数
###
epochs = 1000
batch_size = 128
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

###
# 作成したモデル(脳みそ)の性能を評価します。
###
print()
score = model.evaluate(x_test, y_test, verbose=1)
print('■作成したモデルの性能評価')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print()

###
# 最後に「株主総会」という単語を含む文章を想定して、人工知能にカテゴリを推測させます。
# texts を tokenizer で matrix というベクトル化表現に変形させ、
# model.predict() で「推測」を実行します。
# result には、各ラベルごとの「確率」が数列で格納されます。 
###
print('■作成したモデルを実際に利用してみる')
texts = ["株主総会"]
matrix = tokenizer.texts_to_matrix(texts, "binary")
result = model.predict(matrix)
print("入力:", texts)
print(label_tokenizer.word_index)
print(result)


<script src="https://gist.github.com/oscasierra/00d57d53ddaade973bf46a1450852acb.js"></script>

## ライブラリのインポート

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import sys
import os
import heapq
import glob
import collections

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, InputLayer
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


## 教師データの読み込み

In [2]:
# 読み込むファイルの名前
file = "data.txt"

# 各行のラベルを保持するリスト
labels = []

# 各行のデータ部を保持するリスト
words_arr  = []

with open(file) as f:
  for line in f:

    # 行をカンマで分割し、ラベル部とデータ部を分割します
    blocks = line.rstrip().split(',',1)

    # ラベル部, データ部 それぞれをリストに追加します
    labels.append(blocks[0])
    words_arr.append(blocks[1])

print(len(labels),"行分のデータを読み込みました。")

55 行分のデータを読み込みました。


## 文章のベクトル化
文章をベクトル化するためには、各単語に対してベクトル上の位置を定める必要があります。 そのためには、各単語にユニークなIDを割り当てる必要があり、その割り当てのための辞書を word_dict として定義します。

In [3]:
# データ部をベクトル化します
tokenizer = Tokenizer()
tokenizer.fit_on_texts(words_arr)
x_data = tokenizer.texts_to_matrix(words_arr, "binary")

# ラベル部をベクトル化します
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
y_data = label_tokenizer.texts_to_matrix(labels, "binary")

print("データ内の各単語は、以下のインデックス番号に紐づけられました。")
print(tokenizer.word_index)
print()

print("データ内のラベルは、以下のインデックス番号に紐づけられました。")
print(label_tokenizer.word_index)
print()

print("各行のデータ部は次のようにベクトル化されました (先頭の3個だけを表示)")
print(x_data[:5])
print()

print("各行のラベル部は次のようにリスト化しています。 ")
print(y_data[:5])

データ内の各単語は、以下のインデックス番号に紐づけられました。
{'apple': 1, '天気': 2, '人身事故': 3, '運転': 4, '雨': 5, '雪': 6, '列車': 7, '都心': 8, 'iphone': 9, '東京': 10, '運転再開': 11, '可能性': 12, '関東': 13, '影響': 14, '再開': 15, '開催': 16, '一部': 17, '運転見合わせ': 18, '各地': 19, '冷え込み': 20, '氷点下': 21, 'ces': 22, '雲': 23, '見合わせ': 24, '太平洋側': 25, '広く': 26, '晴天': 27, '全国的': 28, '発表': 29, 'ダイヤ': 30, '乱れ': 31, '九州': 32, '遅れ': 33, '反発': 34, '一時': 35, '熊本電鉄': 36, '脱線': 37, '区間': 38, 'この': 39, '冬': 40, 'いちばん': 41, '業績': 42, '下方修正': 43, '買い替え': 44, '本日': 45, '晴れ': 46, '最高気温': 47, '26度': 48, '午前': 49, '日経平均株価': 50, '終値': 51, '20000円': 52, '会場': 53, '巨大': 54, '広告': 55, 'グーグル': 56, '挑発': 57, '現在': 58, '山手線': 59, '外回り': 60, '遅延': 61, '5分': 62, '成人式': 63, 'どんと祭': 64, '3連休': 65, '東北': 66, 'store': 67, '都内': 68, '5店舗': 69, '近鉄大阪線': 70, '週末': 71, '初雪': 72, '健康分野': 73, '新サービス': 74, '青空': 75, '多い': 76, '景気': 77, '拡大': 78, '局面': 79, '終わり': 80, '年前半': 81, '下値': 82, '1万8000円': 83, '小沢氏': 84, '京王井の頭線': 85, '東松原': 86, '明大前': 87, '駅間': 88, '踏切': 89, '点検': 90, '

## 訓練データと検証データの準備
続いて学習用データを「訓練データ」と「検証データ」に分けます。 ここでは、9:1の割合に分けます。

| 変数名 | 説明 |
|:--------|:------|
| x_train | 訓練データ (データ部) |
| y_train | 訓練データ (ラベル部) |
| x_test | 検証データ (データ部) |
| y_test | 検証データ (ラベル部) |

In [4]:
#split
train_size = int(len(x_data) * 0.9)
x_train,x_test = x_data[:train_size],x_data[train_size:]
y_train,y_test = y_data[:train_size],y_data[train_size:]
print("x_train.shape : ", x_train.shape)
print("x_test.shape  : ", x_test.shape)
print("y_train.shape : ", y_train.shape)
print("y_test.shape  : ", y_test.shape)

x_train.shape :  (49, 252)
x_test.shape  :  (6, 252)
y_train.shape :  (49, 5)
y_test.shape  :  (6, 5)


## モデルの構築
　さて、ここからがやっと人工知能をつくるパートです。 まずは人工知能のモデルを作成します。 作りたいモデルの仕様は次のようなものです。

- 入力の数はデータベクトルの長さ。
- 出力の数はラベルベクトルの長さ。

ここでは、次のようにモデルを構築します。

In [5]:
# Dense : 第1引数で出力の次元数を指定します。入力の次元数はinput_shapeで指定します(指定しない場合は出力と同じ)
model = Sequential()
model.add(InputLayer(input_shape=(x_train.shape[1],)))
model.add(Dense(y_train.shape[1], activation='softmax'))

# モデルをコンパイルする
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

## 学習
　モデルを作り終えたら、ついに本題の「学習」になります。 学習のフェーズで重要なのは、学習回数です。 簡単に例えるなら「1冊の学習ドリルを何回学習したか」みたいな感じです。 人工知能も同じ問題を何度も反復することで賢くなります。 学習用データ(x_train, y_train)を何度もモデルに入力することが大切です。

In [6]:
epochs = 800
batch_size = 128
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

print("leaning done")

Train on 49 samples, validate on 6 samples
Epoch 1/800
49/49 [==============================] - 0s 10ms/step - loss: 1.6043 - acc: 0.1837 - val_loss: 1.6391 - val_acc: 0.1667
Epoch 2/800
49/49 [==============================] - 0s 75us/step - loss: 1.5733 - acc: 0.2653 - val_loss: 1.6309 - val_acc: 0.1667
Epoch 3/800
49/49 [==============================] - 0s 72us/step - loss: 1.5511 - acc: 0.2653 - val_loss: 1.6241 - val_acc: 0.1667
Epoch 4/800
49/49 [==============================] - 0s 65us/step - loss: 1.5326 - acc: 0.2653 - val_loss: 1.6181 - val_acc: 0.1667
Epoch 5/800
49/49 [==============================] - 0s 90us/step - loss: 1.5164 - acc: 0.2857 - val_loss: 1.6126 - val_acc: 0.1667
Epoch 6/800
49/49 [==============================] - 0s 70us/step - loss: 1.5016 - acc: 0.3061 - val_loss: 1.6074 - val_acc: 0.1667
Epoch 7/800
49/49 [==============================] - 0s 76us/step - loss: 1.4878 - acc: 0.3061 - val_loss: 1.6026 - val_acc: 0.1667
Epoch 8/800
49/49 [==============

Epoch 63/800
49/49 [==============================] - 0s 57us/step - loss: 1.0011 - acc: 1.0000 - val_loss: 1.4090 - val_acc: 0.6667
Epoch 64/800
49/49 [==============================] - 0s 47us/step - loss: 0.9940 - acc: 1.0000 - val_loss: 1.4058 - val_acc: 0.6667
Epoch 65/800
49/49 [==============================] - 0s 55us/step - loss: 0.9869 - acc: 1.0000 - val_loss: 1.4027 - val_acc: 0.6667
Epoch 66/800
49/49 [==============================] - 0s 59us/step - loss: 0.9799 - acc: 1.0000 - val_loss: 1.3996 - val_acc: 0.6667
Epoch 67/800
49/49 [==============================] - 0s 54us/step - loss: 0.9729 - acc: 1.0000 - val_loss: 1.3965 - val_acc: 0.6667
Epoch 68/800
49/49 [==============================] - 0s 58us/step - loss: 0.9659 - acc: 1.0000 - val_loss: 1.3934 - val_acc: 0.6667
Epoch 69/800
49/49 [==============================] - 0s 62us/step - loss: 0.9590 - acc: 1.0000 - val_loss: 1.3903 - val_acc: 0.6667
Epoch 70/800
49/49 [==============================] - 0s 60us/step - 

Epoch 125/800
49/49 [==============================] - 0s 66us/step - loss: 0.6324 - acc: 1.0000 - val_loss: 1.2261 - val_acc: 0.8333
Epoch 126/800
49/49 [==============================] - 0s 52us/step - loss: 0.6276 - acc: 1.0000 - val_loss: 1.2233 - val_acc: 0.8333
Epoch 127/800
49/49 [==============================] - 0s 59us/step - loss: 0.6228 - acc: 1.0000 - val_loss: 1.2206 - val_acc: 0.8333
Epoch 128/800
49/49 [==============================] - 0s 51us/step - loss: 0.6181 - acc: 1.0000 - val_loss: 1.2178 - val_acc: 0.8333
Epoch 129/800
49/49 [==============================] - 0s 52us/step - loss: 0.6134 - acc: 1.0000 - val_loss: 1.2150 - val_acc: 0.8333
Epoch 130/800
49/49 [==============================] - 0s 61us/step - loss: 0.6087 - acc: 1.0000 - val_loss: 1.2123 - val_acc: 0.8333
Epoch 131/800
49/49 [==============================] - 0s 48us/step - loss: 0.6040 - acc: 1.0000 - val_loss: 1.2096 - val_acc: 0.8333
Epoch 132/800
49/49 [==============================] - 0s 53us

Epoch 187/800
49/49 [==============================] - 0s 57us/step - loss: 0.3908 - acc: 1.0000 - val_loss: 1.0653 - val_acc: 0.8333
Epoch 188/800
49/49 [==============================] - 0s 62us/step - loss: 0.3878 - acc: 1.0000 - val_loss: 1.0629 - val_acc: 0.8333
Epoch 189/800
49/49 [==============================] - 0s 58us/step - loss: 0.3847 - acc: 1.0000 - val_loss: 1.0605 - val_acc: 0.8333
Epoch 190/800
49/49 [==============================] - 0s 57us/step - loss: 0.3817 - acc: 1.0000 - val_loss: 1.0581 - val_acc: 0.8333
Epoch 191/800
49/49 [==============================] - 0s 53us/step - loss: 0.3787 - acc: 1.0000 - val_loss: 1.0557 - val_acc: 0.8333
Epoch 192/800
49/49 [==============================] - 0s 57us/step - loss: 0.3758 - acc: 1.0000 - val_loss: 1.0533 - val_acc: 0.8333
Epoch 193/800
49/49 [==============================] - 0s 51us/step - loss: 0.3728 - acc: 1.0000 - val_loss: 1.0510 - val_acc: 0.8333
Epoch 194/800
49/49 [==============================] - 0s 55us

Epoch 249/800
49/49 [==============================] - 0s 57us/step - loss: 0.2397 - acc: 1.0000 - val_loss: 0.9325 - val_acc: 0.8333
Epoch 250/800
49/49 [==============================] - 0s 56us/step - loss: 0.2378 - acc: 1.0000 - val_loss: 0.9305 - val_acc: 0.8333
Epoch 251/800
49/49 [==============================] - 0s 55us/step - loss: 0.2360 - acc: 1.0000 - val_loss: 0.9286 - val_acc: 0.8333
Epoch 252/800
49/49 [==============================] - 0s 60us/step - loss: 0.2341 - acc: 1.0000 - val_loss: 0.9267 - val_acc: 0.8333
Epoch 253/800
49/49 [==============================] - 0s 56us/step - loss: 0.2323 - acc: 1.0000 - val_loss: 0.9248 - val_acc: 0.8333
Epoch 254/800
49/49 [==============================] - 0s 57us/step - loss: 0.2304 - acc: 1.0000 - val_loss: 0.9228 - val_acc: 0.8333
Epoch 255/800
49/49 [==============================] - 0s 55us/step - loss: 0.2286 - acc: 1.0000 - val_loss: 0.9209 - val_acc: 0.8333
Epoch 256/800
49/49 [==============================] - 0s 50us

...

Epoch 497/800
49/49 [==============================] - 0s 52us/step - loss: 0.0342 - acc: 1.0000 - val_loss: 0.5573 - val_acc: 1.0000
Epoch 498/800
49/49 [==============================] - 0s 47us/step - loss: 0.0340 - acc: 1.0000 - val_loss: 0.5563 - val_acc: 1.0000
Epoch 499/800
49/49 [==============================] - 0s 51us/step - loss: 0.0337 - acc: 1.0000 - val_loss: 0.5550 - val_acc: 1.0000
Epoch 500/800
49/49 [==============================] - 0s 58us/step - loss: 0.0334 - acc: 1.0000 - val_loss: 0.5540 - val_acc: 1.0000
Epoch 501/800
49/49 [==============================] - 0s 44us/step - loss: 0.0332 - acc: 1.0000 - val_loss: 0.5527 - val_acc: 1.0000
Epoch 502/800
49/49 [==============================] - 0s 52us/step - loss: 0.0329 - acc: 1.0000 - val_loss: 0.5517 - val_acc: 1.0000
Epoch 503/800
49/49 [==============================] - 0s 58us/step - loss: 0.0327 - acc: 1.0000 - val_loss: 0.5505 - val_acc: 1.0000
Epoch 504/800
49/49 [==============================] - 0s 53us

Epoch 559/800
49/49 [==============================] - 0s 55us/step - loss: 0.0213 - acc: 1.0000 - val_loss: 0.4910 - val_acc: 1.0000
Epoch 560/800
49/49 [==============================] - 0s 51us/step - loss: 0.0211 - acc: 1.0000 - val_loss: 0.4901 - val_acc: 1.0000
Epoch 561/800
49/49 [==============================] - 0s 57us/step - loss: 0.0209 - acc: 1.0000 - val_loss: 0.4890 - val_acc: 1.0000
Epoch 562/800
49/49 [==============================] - 0s 46us/step - loss: 0.0208 - acc: 1.0000 - val_loss: 0.4882 - val_acc: 1.0000
Epoch 563/800
49/49 [==============================] - 0s 70us/step - loss: 0.0206 - acc: 1.0000 - val_loss: 0.4871 - val_acc: 1.0000
Epoch 564/800
49/49 [==============================] - 0s 53us/step - loss: 0.0205 - acc: 1.0000 - val_loss: 0.4862 - val_acc: 1.0000
Epoch 565/800
49/49 [==============================] - 0s 43us/step - loss: 0.0203 - acc: 1.0000 - val_loss: 0.4851 - val_acc: 1.0000
Epoch 566/800
49/49 [==============================] - 0s 61us

Epoch 621/800
49/49 [==============================] - 0s 56us/step - loss: 0.0133 - acc: 1.0000 - val_loss: 0.4337 - val_acc: 1.0000
Epoch 622/800
49/49 [==============================] - 0s 47us/step - loss: 0.0132 - acc: 1.0000 - val_loss: 0.4329 - val_acc: 1.0000
Epoch 623/800
49/49 [==============================] - 0s 48us/step - loss: 0.0131 - acc: 1.0000 - val_loss: 0.4319 - val_acc: 1.0000
Epoch 624/800
49/49 [==============================] - 0s 61us/step - loss: 0.0130 - acc: 1.0000 - val_loss: 0.4312 - val_acc: 1.0000
Epoch 625/800
49/49 [==============================] - 0s 58us/step - loss: 0.0129 - acc: 1.0000 - val_loss: 0.4302 - val_acc: 1.0000
Epoch 626/800
49/49 [==============================] - 0s 50us/step - loss: 0.0128 - acc: 1.0000 - val_loss: 0.4295 - val_acc: 1.0000
Epoch 627/800
49/49 [==============================] - 0s 46us/step - loss: 0.0127 - acc: 1.0000 - val_loss: 0.4285 - val_acc: 1.0000
Epoch 628/800
49/49 [==============================] - 0s 51us

Epoch 683/800
49/49 [==============================] - 0s 63us/step - loss: 0.0083 - acc: 1.0000 - val_loss: 0.3841 - val_acc: 1.0000
Epoch 684/800
49/49 [==============================] - 0s 49us/step - loss: 0.0083 - acc: 1.0000 - val_loss: 0.3834 - val_acc: 1.0000
Epoch 685/800
49/49 [==============================] - 0s 42us/step - loss: 0.0082 - acc: 1.0000 - val_loss: 0.3826 - val_acc: 1.0000
Epoch 686/800
49/49 [==============================] - 0s 50us/step - loss: 0.0082 - acc: 1.0000 - val_loss: 0.3820 - val_acc: 1.0000
Epoch 687/800
49/49 [==============================] - 0s 46us/step - loss: 0.0081 - acc: 1.0000 - val_loss: 0.3811 - val_acc: 1.0000
Epoch 688/800
49/49 [==============================] - 0s 62us/step - loss: 0.0080 - acc: 1.0000 - val_loss: 0.3805 - val_acc: 1.0000
Epoch 689/800
49/49 [==============================] - 0s 63us/step - loss: 0.0080 - acc: 1.0000 - val_loss: 0.3797 - val_acc: 1.0000
Epoch 690/800
49/49 [==============================] - 0s 46us

Epoch 745/800
49/49 [==============================] - 0s 47us/step - loss: 0.0053 - acc: 1.0000 - val_loss: 0.3412 - val_acc: 1.0000
Epoch 746/800
49/49 [==============================] - 0s 50us/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.3406 - val_acc: 1.0000
Epoch 747/800
49/49 [==============================] - 0s 54us/step - loss: 0.0052 - acc: 1.0000 - val_loss: 0.3399 - val_acc: 1.0000
Epoch 748/800
49/49 [==============================] - 0s 48us/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.3393 - val_acc: 1.0000
Epoch 749/800
49/49 [==============================] - 0s 60us/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.3386 - val_acc: 1.0000
Epoch 750/800
49/49 [==============================] - 0s 57us/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.3380 - val_acc: 1.0000
Epoch 751/800
49/49 [==============================] - 0s 47us/step - loss: 0.0050 - acc: 1.0000 - val_loss: 0.3373 - val_acc: 1.0000
Epoch 752/800
49/49 [==============================] - 0s 45us

## 評価検証
　それでは最後に、出来上がった人工知能モデルが、どれだけ優秀なのかを評価しましょう。 出来上がった人工知能モデルに検証用データ (x_test と y_test )を与えて、正解率を求めて評価します。

In [7]:
score = model.evaluate(x_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

6/6 [==============================] - 0s 251us/step

Test loss: 0.3078007400035858
Test accuracy: 1.0


accuracy とは日本語で「正確さ」という意味で、今回の場合であれば「文章を正しいカテゴリに分類できる確率」を表しています。

## モデルの利用
　それでは完成したモデルを利用してみましょう。  下の例では「株主総会」という文章(texts)をベクトル化(matrix)して、model.predict() に与えています。 モデルが推測した結果は result という変数で、result には各ラベルごとの結果が格納されています。 (0番目の要素は、どれにも当てはまらなかった確率)

In [8]:
texts = ["株主総会"]
matrix = tokenizer.texts_to_matrix(texts, "binary")
result = model.predict(matrix)
print(label_tokenizer.word_index)
print(result)

{'天気': 1, 'テクノロジ': 2, '交通': 3, '経済': 4}
[[0.0382612  0.10484572 0.6289686  0.14879456 0.07912987]]


## APPENDIX
本チュートリアルのために作成した仮装環境の構築コマンドを参考までに掲載しておきます。 Anaconda を利用し、以下のコマンドで仮想環境を構築しました。

    conda create -n ai-study python=3.6 python keras jupyter matplotlib
    conda activate ai-study
